In [ ]:
import urllib.request, json
import pandas as pd

try:
    url = "https://api.nationaltransport.ie/gtfsr/v1?format=json"

    hdr ={
    # Request headers
    'Cache-Control': 'no-cache',
    'x-api-key': 'd40a65c06e5d4370925e57326cba2bb5',
    }

    req = urllib.request.Request(url, headers=hdr)

    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
   # print(response.getcode())
   # print(response.read())
    elevations = response.read()
    data = json.loads(elevations)
    df = pd.json_normalize(data, sep=',')
except Exception as e:
    print(e)

In [ ]:
data.keys()

dict_keys(['Header', 'Entity'])

In [ ]:
type(data['Entity'])

list

In [ ]:
data['Entity'][0]['TripUpdate']['StopTimeUpdate'][0].keys()

dict_keys(['StopSequence', 'StopId', 'Departure', 'ScheduleRelationship'])

In [ ]:
data['Header'].keys()

dict_keys(['GtfsRealtimeVersion', 'Incrementality', 'Timestamp'])

In [ ]:
tripupdates=data['Entity']

In [ ]:
seen_stops = set()
for tripupdate in tripupdates:
  routeid = tripupdate['TripUpdate']['Trip']['RouteId']
  starttime = tripupdate['TripUpdate']['Trip']['StartTime']
  #vehicle = tripupdate['VehiclePosition']['VehicleDescriptor']
  
  if '46A'in routeid:
    #print(routeid)
    try:
      stops = tripupdate['TripUpdate']['StopTimeUpdate']
    except KeyError:
      pass
    else:
      print('------------')
      print(routeid, starttime, len(stops))
      
      for stop in stops:
        try:
          delay = stop['Arrival']['Delay']
          stopid = stop['StopId']
        except KeyError:
          print('No data')
        else:
          print(stopid, delay, stopid in seen_stops)
          if delay >=180:
             print('Bus delayed, enable slow down protocol for next bus leaving after',starttime)
          elif delay <=-180:
            print('Bus early, enable slow down protocol',starttime)
          seen_stops.add(stopid)
      #if delay >=120:
       # print('Bus delayed, enable slow down protocol for next bus leaving after',starttime)


------------
60-46A-b12-1 09:10:00 16
No data
8250DB004568 -60 False
8250DB002056 0 False
8250DB002064 -60 False
8250DB002065 -120 False
8250DB004727 -60 False
8250DB002069 -120 False
8250DB002070 -60 False
8250DB000768 -120 False
8220DB000906 -180 False
Bus early, enable slow down protocol 09:10:00
8220DB000002 -120 False
8220DB000192 -180 False
Bus early, enable slow down protocol 09:10:00
8220DB000797 -120 False
8220DB000801 -180 False
Bus early, enable slow down protocol 09:10:00
8220DB000802 -120 False
8220DB000807 -180 False
Bus early, enable slow down protocol 09:10:00
------------
60-46A-b12-1 09:30:00 7
No data
8250DB002042 120 False
8250DB004568 60 True
8250DB002056 180 True
Bus delayed, enable slow down protocol for next bus leaving after 09:30:00
8250DB002062 120 False
8250DB002070 60 True
8250DB000768 120 True
------------
60-46A-b12-1 09:45:00 7
No data
8250DB004568 -120 True
8250DB002060 -180 False
Bus early, enable slow down protocol 09:45:00
8250DB002061 -240 False
Bus

In [ ]:
seen_stops = set()
slowlist=[]
fastlist=[]
for tripupdate in tripupdates:
  routeid = tripupdate['TripUpdate']['Trip']['RouteId']
  starttime = tripupdate['TripUpdate']['Trip']['StartTime']
  #vehicle = tripupdate['VehiclePosition']['VehicleDescriptor']
  
  if '46A'in routeid:
    #print(routeid)
    try:
      stops = tripupdate['TripUpdate']['StopTimeUpdate']
    except KeyError:
      pass
    else:
      print('------------')
      print(routeid, starttime, len(stops))
      
      for stop in stops:
        try:
          delay = stop['Arrival']['Delay']
          stopid = stop['StopId']
        except KeyError:
          print('No data')
        else:
          print(stopid, delay, stopid in seen_stops)
          if delay >=180:
             print('Bus delayed, enable slow down protocol for next bus leaving after',starttime)
             slowlist.append(starttime)
             slowlist.append(delay)
             slowlist.append(stopid)
          elif delay <=-180:
            print('Bus early, enable slow down protocol',starttime)
            fastlist.append(starttime)
            fastlist.append(delay)
            fastlist.append(stopid)
          seen_stops.add(stopid)

      #if delay >=120:
       # print('Bus delayed, enable slow down protocol for next bus leaving after',starttime)


------------
60-46A-b12-1 09:56:00 13
No data
8250DB002040 60 False
8250DB002042 120 False
8250DB004568 60 False
8250DB002056 0 False
8250DB002057 -60 False
8250DB002061 0 False
8250DB002062 -60 False
8250DB000461 0 False
8250DB002070 -60 False
8220DB000770 0 False
8220DB000774 -60 False
8220DB000776 0 False
------------
60-46A-b12-1 10:04:00 6
No data
8250DB004568 0 True
8250DB002056 -120 True
8250DB002060 -180 False
Bus early, enable slow down protocol 10:04:00
8250DB002069 -240 False
Bus early, enable slow down protocol 10:04:00
8220DB000775 -300 False
Bus early, enable slow down protocol 10:04:00
------------
60-46A-b12-1 10:08:00 1
No data
------------
60-46A-b12-1 10:12:00 13
No data
8250DB004568 -60 True
8250DB002056 -120 True
8250DB004727 -180 False
Bus early, enable slow down protocol 10:12:00
8250DB002068 -240 False
Bus early, enable slow down protocol 10:12:00
8250DB002069 -300 True
Bus early, enable slow down protocol 10:12:00
8250DB002070 -180 True
Bus early, enable slow d

In [ ]:

for x in range(len(slowlist)):
  print(slowlist[x])

print('---------')

for x in range(len(fastlist)):
  print(fastlist[x])


10:36:00
180
8250DB002040
10:36:00
240
8250DB002041
10:36:00
300
8250DB002043
10:36:00
240
8250DB002046
10:36:00
300
8250DB004568
10:36:00
360
8250DB002056
10:36:00
300
8250DB002057
10:36:00
240
8250DB002062
10:36:00
180
8250DB004727
10:36:00
240
8250DB000461
10:36:00
360
8250DB002068
10:52:00
180
8250DB002058
10:52:00
180
8250DB002062
---------
10:04:00
-180
8250DB002060
10:04:00
-240
8250DB002069
10:04:00
-300
8220DB000775
10:12:00
-180
8250DB004727
10:12:00
-240
8250DB002068
10:12:00
-300
8250DB002069
10:12:00
-180
8250DB002070
10:12:00
-240
8250DB000768
10:12:00
-300
8220DB000775
10:12:00
-360
8220DB000907
10:12:00
-300
8220DB000909
10:12:00
-360
8220DB000786
10:12:00
-300
8220DB000792
10:28:00
-180
8250DB000768
10:28:00
-240
8220DB000769
10:28:00
-300
8220DB000775
10:28:00
-360
8220DB000777
10:28:00
-420
8220DB000906


In [ ]:
class protocol:
  def getdata():
    import urllib.request, json
    import pandas as pd

    try:
      url = "https://api.nationaltransport.ie/gtfsr/v1?format=json"

      hdr ={
      # Request headers
      'Cache-Control': 'no-cache',
      'x-api-key': 'd40a65c06e5d4370925e57326cba2bb5',
      }

      req = urllib.request.Request(url, headers=hdr)

      req.get_method = lambda: 'GET'
      response = urllib.request.urlopen(req)
   # print(response.getcode())
   # print(response.read())
      elevations = response.read()
      data = json.loads(elevations)
      df = pd.json_normalize(data, sep=',')
    except Exception as e:
      print(e)

  def slowbus():
    
    seen_stops = set()
    for tripupdate in tripupdates:
      routeid = tripupdate['TripUpdate']['Trip']['RouteId']
      starttime = tripupdate['TripUpdate']['Trip']['StartTime']
      #vehicle = tripupdate['VehiclePosition']['VehicleDescriptor']
    
    if '46A'in routeid:
    #print(routeid)
      try:
        stops = tripupdate['TripUpdate']['StopTimeUpdate']
      except KeyError:
        pass
      else:
        print('------------')
        print(routeid, starttime, len(stops))
      
        for stop in stops:
          try:
            delay = stop['Arrival']['Delay']
            stopid = stop['StopId']
          except KeyError:
            print('No data')
          else:
            print(stopid, delay, stopid in seen_stops)
            if delay >=120:
               print('Bus delayed, enable slow down protocol for next bus leaving after',starttime)
          seen_stops.add(stopid)
      #if delay >=120:
       # print('Bus delayed, enable slow down protocol for next bus leaving after',starttime)


In [ ]:
protocol.getdata()
protocol.slowbus()

Hello
